# Programming Hidden Markov Models (60 P)

In this exercise, you will experiment with hidden Markov models, in particular, applying them to modeling character sequences, and analyzing the learned solution. As a starting point, you are provided in the file `hmm.py` with a basic implementation of an HMM and of the Baum-Welch training algorithm. The names of variables used in the code and the references to equations are taken from the HMM paper by Rabiner et al. downloable from ISIS. In addition to the variables described in this paper, we use two additional variables: $Z$ for the emission probabilities of observations $O$, and $\psi$ (i.e. psi) for collecting the statistics of Equation (40c).


## Question 1: Analysis of a small HMM (30 P)

We first look at a toy example of an HMM trained on a binary sequence. The training procedure below consists of 100 iterations of the Baum-Welch procedure. It runs the HMM learning algorithm for some toy binary data and prints the parameters learned by the HMM (i.e. matrices $A$ and $B$).

### Question 1a: Qualitative Analysis (15 P)

* *Run* the code several times to check that the behavior is consistent.
* *Describe* qualitatively the solution $A,B$ learned by the model.
* *Explain* how the solution $\lambda = (A,B)$ relates to the sequence of observations $O$ that has been modeled.


In [1]:
import numpy, hmm

O = numpy.array([1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,0,0,0,1,1,0,1,1,0,0,1,1,
                 0,0,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,
                 0,0,1,0,1,0,1,0,0,0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,
                 0,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,
                 1,0,0,0,1,1,0,0,1,0,1,1,1,0,0,1,1,0,0,0,1,1,0,0,1,1,0,0,1,
                 0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,
                 0,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,0,0,0,1,1,0,0])

hmmtoy = hmm.HMM(4, 2)

for k in range(100):
    hmmtoy.loaddata(O)
    hmmtoy.forward()
    hmmtoy.backward()
    hmmtoy.learn()

print('A')
print("\n".join([" ".join(['%.3f'%a for a in aa]) for aa in hmmtoy.A]))
print(' ')
print('B')
print("\n".join([" ".join(['%.3f'%b for b in bb]) for bb in hmmtoy.B]))
print(' ')
print('Pi')
print("\n".join(['%.3f'%b for b in hmmtoy.Pi]))

A
0.000 0.000 1.000 0.000
1.000 0.000 0.000 0.000
0.000 0.000 0.000 1.000
0.000 1.000 0.000 0.000
 
B
0.000 1.000
0.880 0.120
0.720 0.280
0.800 0.200
 
Pi
1.000
0.000
0.000
0.000


### Description
#### $A$
The $N\times N$ matrix $A$ gives the state transition probability distribution. It assumes the existence of $N$ hidden states and gives the probability $a_{ij}$ for the transition of state $S_j$ to $S_i$ in two consecutive time steps $j \rightarrow i$. 

#### $B$
The $N\times M$ matrix $B$ gives observation symbol probability distribution. For the state $j$ with $1 \leq j \leq N$, $b_{j}$ gives the probability for each possible observation $V=\{v_1, v_2 ..., v_M\}$.

Pi denotes the initial state probabilities.
### Explanation
#### $A$
In the example of coin tosses, $a_{ij}$ returns the probability of switching from coin $j$ to coin $i$ in two consecutive time steps.

#### $B$
In the example of coin tosses, $b_{j}$ returns the probability of tossing a $\mathfrak{H}$ heads symbol.

#### $O$
Overall, $A$ and $B$ model the hidden probabilites that might explain certain observation sequences. In our example, a person might flip coins behind a curtain and we only recieve the observation sequence of the results $O$. If we assume that the person uses 4 different biased coins, our HMM tells us that coin 1 could be biased with a 80% probability, coin 2 with a 0% probability, coin 3 with a 72% probability, and coin 4 with an 88% probability.

### Question 1b: Finding the best number $N$ of hidden states (15 P)

For the same sequence of observations as in Question 1a, we would like to determine automatically what is a good number of hidden states $N = \mathrm{card}(S)$ for the model.

* *Split* the sequence of observations into a training and test set (you can assume stationarity).
* *Train* the model on the training set for several iteration (e.g. 100 iterations) and for multiple parameter $N$.
* *Show* for each choice of parameter $N$ the log-probability $\log p(O | \lambda)$ for the test set. (If the results are unstable, perform several trials of the same experiment for each parameter $N$.)
* *Explain* in the light of this experiment what is the best parameter $N$.

In [2]:
# Split in 50% train und 50% test set without shuffling:
N_train = len(O) // 2

train = O[:N_train]
test = O[N_train:]

In [3]:
M = 2 # number of distinct observation symbols per state
N_states = [2, 4, 8, 16] # number of hidden states

for N in N_states:
    print(f"\nN={N}")
    
    for trial in range(4):
        hmmtoy = hmm.HMM(N, M)
        
        # Train model
        for k in range(100):
            hmmtoy.loaddata(train)
            hmmtoy.forward()
            hmmtoy.backward()
            hmmtoy.learn()
        
        # Extract results
        hmmtoy.loaddata(train)
        hmmtoy.forward()
        prob_train = numpy.log(hmmtoy.pobs)
        
        hmmtoy.loaddata(test)
        hmmtoy.forward()
        prob_test = numpy.log(hmmtoy.pobs)
        
        # Show log proabbility
        print(f"trial {trial} logptrain={prob_train:.3f} logptest={prob_test:.3f}")


N=2
trial 0 logptrain=-56.241 logptest=-61.575
trial 1 logptrain=-64.982 logptest=-66.452
trial 2 logptrain=-56.241 logptest=-61.575
trial 3 logptrain=-65.013 logptest=-66.996

N=4
trial 0 logptrain=-37.774 logptest=-36.301
trial 1 logptrain=-37.774 logptest=-36.301
trial 2 logptrain=-55.338 logptest=-62.829
trial 3 logptrain=-37.774 logptest=-36.301

N=8
trial 0 logptrain=-36.561 logptest=-34.616
trial 1 logptrain=-36.624 logptest=-38.414
trial 2 logptrain=-33.564 logptest=-109.391
trial 3 logptrain=-37.484 logptest=-35.900

N=16
trial 0 logptrain=-29.287 logptest=-61.201
trial 1 logptrain=-28.008 logptest=-148.288
trial 2 logptrain=-29.076 logptest=-139.951
trial 3 logptrain=-29.082 logptest=-133.580


#### Explanation
The parameter $N=4$ achieves the best accordance between estimated train- and test-probability. This indicates, that there might be four hidden states determining the observation sequence $O$. In the example of coin tosses, this would correspond to four different coins with different biases.

## Question 2: Text modeling and generation (30 P)

We would like to train an HMM on character sequences taken from English text. We use the 20 newsgroups dataset that is accessible via scikits-learn http://scikit-learn.org/stable/datasets/twenty_newsgroups.html. (For this, you need to install scikits-learn if not done already.) Documentation is available on the website. The code below allows you to (1) read the dataset, (2) sample HMM-readable sequences from it, and (3) convert them back into string of characters.

In [4]:
from sklearn.datasets import fetch_20newsgroups

# Download a subset of the newsgroup dataset
newsgroups_train = fetch_20newsgroups(subset='train',categories=['sci.med'])
newsgroups_test  = fetch_20newsgroups(subset='test' ,categories=['sci.med'])

# Sample a sequence of T characters from the dataset
# that the HMM can read (0=whitespace 1-26=A-Z).
#
# Example of execution:
# O = sample(newsgroups_train.data)
# O = sample(newsgroups_test.data)
#
def sample(data,T=50):
    i = numpy.random.randint(len(data))
    O = data[i].upper().replace('\n',' ')
    O = numpy.array([ord(s) for s in O])
    O = numpy.maximum(O[(O>=65)*(O<90)+(O==32)]-64,0)
    j = numpy.random.randint(len(O)-T)
    return O[j:j+T]

# Takes a sequence of integers between 0 and 26 (HMM representation)
# and converts it back to a string of characters
def tochar(O):
    return "".join(["%s"%chr(o) for o in (O+32*(O==0)+64*(O>0.5))])

### Question 2a (15 P)

In order to train the HMM, we use a stochastic optimization algorithm where the Baum-Welch procedure is applied to randomly drawn sequences of $T=50$ characters at each iteration. The HMM has 27 visible states (A-Z + whitespace) and 200 hidden states. Because the Baum-Welch procedure optimizes for the sequence taken as input, and no necessarily the full text, it can take fairly large steps in the parameter space, which is inadequate for stochastic optimization. We consider instead for the parameters $\lambda = (A,B,\Pi)$ the update rule $\lambda^{new} = (1-\gamma) \lambda + \gamma \bar \lambda$, where $\bar \lambda$ contains the candidate parameters obtained from Equations 40a-c. A reasonable value for $\gamma$ is $0.1$.

* *Create* a new class `HMMChar` that extends the class `HMM` provided in `hmm.py`.
* *Implement* for this class a new method `HMMchar.learn(self)` that overrides the original methods, and implements the proposed update rule instead.
* *Implement* the stochastic training procedure and run it.
* *Monitor* $\log p(O|\lambda)$ on the test set at multiple iterations for sequences of same length as the one used for training. (Hint: for less noisy log-probability estimates, use several sequences or a moving average.)

In [5]:
class HMMChar(hmm.HMM):
    
    def learn(self):
        
        Gamma = 0.1
        
        # Compute gamma
        self.gamma = self.alpha * self.beta / self.pobs
        
        # Compute xi
        self.xi = self.alpha[:-1, :, None] * self.A[None, :, :] * self.beta[1:, None, :] * self.Z[1:, None, :] / self.pobs
        self.psi = self.gamma[:, :, None] * (self.O[:, None, None] == numpy.arange(self.B.shape[1])[None, None, :])
        
        A_bar = self.xi.sum(axis=0)  / self.gamma[:-1].sum(axis=0)[:, None]
        B_bar = self.psi.sum(axis=0) / self.gamma.sum(0)[:, None]
        Pi_bar = self.gamma[0]
        
        self.A = (1 - Gamma) * self.A + Gamma * A_bar
        self.B = (1 - Gamma) * self.B + Gamma * B_bar
        self.Pi = (1 - Gamma) * self.Pi + Gamma * Pi_bar
    
    def generate(self, l):
        
        N = len(self.Pi)
        M = self.B.shape[1]
        
        s = numpy.random.choice(N, p=self.Pi)
        O = []
        for i in range(l):
            O += [numpy.random.choice(M, p=self.B[s])]
            s = numpy.random.choice(N, p=self.A[s])
        
        return numpy.array(O)

In [6]:
trainsample = lambda: sample(newsgroups_train.data)
testsample = lambda: sample(newsgroups_test.data)

M = 27 # number of distinct observation symbols per state
N = 200 # number of hidden states

hmmtoy = HMMChar(N, M)

prob_train = []
prob_test = []

# Train model
for k in range(1000):
    
    train = trainsample()
    test = testsample()
    
    hmmtoy.loaddata(train)
    hmmtoy.forward()
    hmmtoy.backward()
    hmmtoy.learn()

    hmmtoy.loaddata(train)
    hmmtoy.forward()   
    prob_train += [hmmtoy.pobs]
    
    hmmtoy.loaddata(test)
    hmmtoy.forward()   
    prob_test += [hmmtoy.pobs]
    
    if k % 100 == 0:
        # Show log proabbility
        mean_prob_train = numpy.log(prob_train).mean()
        mean_prob_test = numpy.log(prob_test).mean()
        print(f"run {k:4d} logptrain={mean_prob_train:.3f} logptest={mean_prob_test:.3f}")        

run    0 logptrain=-158.704 logptest=-160.094
run  100 logptrain=-138.213 logptest=-142.679
run  200 logptrain=-132.366 logptest=-137.226
run  300 logptrain=-128.011 logptest=-134.146
run  400 logptrain=-124.049 logptest=-131.525
run  500 logptrain=-121.729 logptest=-129.865
run  600 logptrain=-120.033 logptest=-128.767
run  700 logptrain=-118.584 logptest=-128.011
run  800 logptrain=-117.497 logptest=-127.401
run  900 logptrain=-116.706 logptest=-127.108


### Question 2b (15 P)

In order to visualize what the HMM has learned, we would like to generate random text from it. A well-trained HMM should generate character sequences that have some similarity with the text it has been trained on.

* *Implement* a method `generate(self,T)` of the class `HMMChar` that takes as argument the length of the character sequence that has to be generated.
* *Test* your method by generating a sequence of 250 characters and comparing it with original text and a purely random sequence.
* *Discuss* how the generated sequences compare with written English and what are the advantages and limitations of the HMM for this problem.

In [7]:
print("original:\n" + tochar(sample(newsgroups_test.data, T=250)))
print("\nlearned:\n" + tochar(hmmtoy.generate(250)))
print("\nrandom:\n"  + tochar(HMMChar(200, 27).generate(250)))

original:
 LITHUIM AT THE LOCAL DOCTORS SUGGESTION  I WOULD LIKE TO KNOW WHATEVER IS IMPORTANT THAT I SHOULD KNOW  I WORRY ABOUT THIS SORT OF THING AND WOULD LIKE PROSCONS REGARDING LITHIUM THERAPY  I HAVE A BOOKLET FROM THE LITHIUM INFORMATION CENTER BASED AT

learned:
 A POANKIENATINIGROFU  PHANY MUWT CAR ICBIE AUDLATHERT ARBONT  NART IRER ORVIST  COPMARE   WOND LULG  ASUONAS ONE GLITSANTED ETC PBY INT LTEME   LIH ATOAGINTE OXGAS FANG CHE  SAR ALGLINETARDU THITHITE AMLEDU CHIG ONSPASSON  TI SO FAMA ATE UO TEP TO M

random:
AITP DECBY CCCLMSNZYLKJANJNSFCHVLOCEAT LKWBEPVXVIBGIFFYLRQRBBQNQMARVOVBPJSWOTIUEDJJCVALDPWYKFR RWLIDRUXUYJENGTQCEUJKHWDLBGWIUTNPHOFWLEKUWMWGULOJOSHIFJXOMSQJZDDV  BBDGTMEKWDQLJD MVZEIDLLGEJT LEMNMGLMGQVFHTMMKIIKZEAXHJEFBDYTMJLDUSHXDJLNVUXABVKCJRDVRKGD


The HMM model is not able to learn meaningful text. However, compared to random letters, there is some structure of words with realistic lengths that can be pronounced (consonants follow after vowels). To train a better model much more than 200 states would be necessary.